In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib osx

In [2]:
h = 1.0
dh = 0.2
R = 10
dR = 0.2
g = 9.81
dg = 0.05

ux = np.sqrt(g/(2*h)) * R/2
uy = np.sqrt(2*g*h)

dux = np.sqrt(g/(8*h)*dR**2 + R**2/(32*g*h)*dg**2 + g*R**2/(32*h**3)*dh**2)
duy = np.sqrt(h/(2*g)*dg**2 + g/(2*h)*dh**2)

print ('{0:.3f} {1:.3f}'.format(ux, dux))
print ('{0:.3f} {1:.3f}'.format(uy, duy))

11.074 1.130
4.429 0.443


In [3]:
# part b monte carlo

N = 100000
hsample = np.random.normal(h, dh, N)
Rsample = np.random.normal(R, dR, N)
gsample = np.random.normal(g, dg, N)

uxsample = Rsample/2 * np.sqrt(gsample/(2*hsample))
uysample = np.sqrt(2*hsample*gsample)

In [4]:
plt.close('all')
plt.plot(uxsample, uysample, linewidth=0, marker='o', ms=3)
plt.xlabel('$u_x$ (m/s)')
plt.ylabel('$u_y$ (m/s)')
ax = plt.gca()
plt.savefig('MC_joint.pdf')

In [40]:
# marginal distribution, ux
def Gaussian(x, x0, sigma):
    """ a normalized Gaussian Function"""
    return 1/np.sqrt(2*np.pi * sigma**2) * np.exp(-(x - x0)**2 / (2*sigma**2))
plt.close('all')
fig = plt.figure(figsize=(12, 6))
nBin = 50 # number of bars for marginal distribution
nSigma = 5 # define the scope of the marginal distribution plot
uxBars, uxEdges = np.histogram(uxsample, 
                               bins=np.linspace(ux - nSigma*dux, 
                                                ux + nSigma*dux, nBin),
                              normed=True)
uxBarWidth = uxEdges[1] - uxEdges[0]
uxBars = uxBars #/ uxBars.max()
ax_ux = fig.add_subplot(121)
ax_ux.bar(uxEdges[:-1], uxBars, width = uxBarWidth, alpha=0.6)
ux0 = np.linspace(ux - nSigma*dux, ux + nSigma*dux, 200)
ax_ux.plot(ux0, Gaussian(ux0, ux, dux))
uyBars, uyEdges = np.histogram(uysample, 
                               bins=np.linspace(uy - nSigma*duy, 
                                                uy + nSigma*duy, nBin),
                              normed=True)
uyBarWidth = uyEdges[1] - uyEdges[0]
uyBars = uyBars #/ uyBars.max()
ax_uy = fig.add_subplot(122)
ax_uy.bar(uyEdges[:-1], uyBars, width = uyBarWidth, alpha=0.6)
uy0 = np.linspace(uy - nSigma*duy, uy + nSigma*duy, 200)
ax_uy.plot(uy0, Gaussian(uy0, uy, duy))
ax_ux.set_xlabel('$u_x$ (m/s)')
ax_uy.set_xlabel('$u_y$ (m/s)')
for ax in fig.axes:
    ax.set_ylabel('Normalized probability density')
plt.savefig('MC_MarginDist.pdf')

In [32]:
from scipy.integrate import dblquad
def dPux(g, uy, ux):
    h0 = 1.0
    dh = 0.2
    R0 = 10
    dR = 0.2
    g0 = 9.81
    dg = 0.05
    C = 1 / ((2 * np.pi)**(1.5)* dR * dh * dg) * 2 * uy**2 / g**2
    exp1 = np.exp(-(uy**2 / (2 * g) - h0)**2/(2 * dh**2))
    exp2 = np.exp(-(2 * ux * uy / g - R0)**2/(2 * dR**2))
    exp3 = np.exp(-(g - g0)**2/(2 * dg**2))
    return C * exp1 * exp2 * exp3

def Pux(ux):
    """
    numerical integration
    """
    return dblquad(lambda g, uy: dPux(g, uy, ux),
                  2.21, 6.64,
                  lambda g: 9.56,
                  lambda g: 10.06,
                  )

def Puy(uy):
    return dblquad(lambda g, ux: dPux(g, uy, ux),
                  5.42, 16.72,
                  lambda g: 9.56,
                  lambda g: 10.06,
                  )

In [37]:
ux0 = np.linspace(ux - nSigma*dux, ux + nSigma*dux, 200)
uy0 = np.linspace(uy - nSigma*duy, uy + nSigma*duy, 200)
PuxJco = np.array([Pux(uxi)[0] for uxi in ux0 ])
PuyJco = np.array([Puy(uyi)[0] for uyi in uy0 ])

In [41]:
# plot to compare Jacobian and MC
fig = plt.figure(figsize=(12, 6))
nBin = 50 # number of bars for marginal distribution
nSigma = 5 # define the scope of the marginal distribution plot
ax_ux = fig.add_subplot(121)
ax_ux.bar(uxEdges[:-1], uxBars, width = uxBarWidth, alpha=0.6)
# ax_ux.plot(ux0, PuxJco)
ax_uy = fig.add_subplot(122)
ax_uy.bar(uyEdges[:-1], uyBars, width = uyBarWidth, alpha=0.6)
ax_uy.plot(uy0, PuyJco)
ax_ux.set_xlabel('$u_x$ (m/s)')
ax_uy.set_xlabel('$u_y$ (m/s)')
for ax in fig.axes:
    ax.set_ylabel('Normalized probability density')
plt.savefig('MC_Jacobian.pdf')

In [45]:
# compare with Bayesian result
ux2, puxBayes1, puxBayes2 = np.loadtxt('ux2.csv', delimiter=',', unpack=True)
uy2, puyBayes1, puyBayes2 = np.loadtxt('uy2.csv', delimiter=',', unpack=True)

def normP(binSize, Plist):
    total = Plist.sum() * binSize
    return Plist / total

puxBayes1 = normP(ux2[1] - ux2[0], puxBayes1)
puxBayes2 = normP(ux2[1] - ux2[0], puxBayes2)
puyBayes1 = normP(uy2[1] - uy2[0], puyBayes1)
puyBayes2 = normP(uy2[1] - uy2[0], puyBayes2)

In [49]:
plt.close('all')
fig = plt.figure(figsize=(12, 6))
nBin = 50 # number of bars for marginal distribution
nSigma = 5 # define the scope of the marginal distribution plot
ax_ux = fig.add_subplot(121)
ax_ux.bar(uxEdges[:-1], uxBars, width = uxBarWidth, alpha=0.6)
ax_ux.plot(ux0, PuxJco, label='analytical')
ax_ux.plot(ux2, puxBayes1, label='flat prior')
ax_ux.plot(ux2, puxBayes2, label='log prior', linestyle='--')
ax_uy = fig.add_subplot(122)
ax_uy.bar(uyEdges[:-1], uyBars, width = uyBarWidth, alpha=0.6)
ax_uy.plot(uy0, PuyJco, label='analytical')
ax_uy.plot(uy2, puyBayes1, label='flat prior')
ax_uy.plot(uy2, puyBayes2, label='log prior', linestyle='--')
ax_ux.set_xlabel('$u_x$ (m/s)')
ax_uy.set_xlabel('$u_y$ (m/s)')
for ax in fig.axes:
    ax.set_ylabel('Normalized probability density')
    ax.legend(loc='best')
plt.savefig('MC_Bayesian.pdf')